In [ ]:
import mysql.connector
import pandas as pd
from dotenv import load_dotenv
import os

# ---------------------------------------
# 1. Connect to MySQL and Fetch Data
# ---------------------------------------
load_dotenv()

MYSQL_HOST = os.getenv("MYSQL_HOST")
MYSQL_USER = os.getenv("MYSQL_USER")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
MYSQL_DATABASE = os.getenv("MYSQL_DATABASE")
MYSQL_PORT = int(os.getenv("MYSQL_PORT"))

cnx = mysql.connector.connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    database=MYSQL_DATABASE,
    port=MYSQL_PORT
)

cursor = cnx.cursor()

# Read all tables into Pandas DataFrames
tables = ["Orders", "customers", "locations", "orderdetails", "regionalmanagers", "returns"]
dataframes = {}

for table in tables:
    query = f"SELECT * FROM {table}"
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [col[0] for col in cursor.description]
    dataframes[table] = pd.DataFrame(rows, columns=columns)

cursor.close()
cnx.close()

# Store tables in separate variables
df_orders = dataframes["Orders"]
df_customers = dataframes["customers"]
df_locations = dataframes["locations"]
df_orderdetails = dataframes["orderdetails"]
df_regionalmanagers = dataframes["regionalmanagers"]
df_returns = dataframes["returns"]

print("✅ Data successfully loaded from MySQL")

# ---------------------------------------
# 2. Data Preprocessing
# ---------------------------------------

# Convert Order Date and Ship Date to datetime
df_orders["Order Date"] = pd.to_datetime(df_orders["Order Date"])
df_orders["Ship Date"] = pd.to_datetime(df_orders["Ship Date"])

# Extract Year from Order Date
df_orders["Year"] = df_orders["Order Date"].dt.year

# Ensure "Returned" column only contains 'Yes'
df_returns["Returned"] = "Yes"

# Fill missing values
df_orders.fillna({"Ship Mode": "Unknown"}, inplace=True)

# ---------------------------------------
# 3. Merge DataFrames
# ---------------------------------------

df_merged = (
    df_orders
    .merge(df_customers, on="Customer ID", how="left")
    .merge(df_orderdetails, on="Order ID", how="left")
    .merge(df_locations, on="Postal Code", how="left")
    .merge(df_regionalmanagers, on="Region", how="left")
    .merge(df_returns, on="Order ID", how="left")
)

df_merged["Returned"].fillna("No", inplace=True)

print("✅ Data successfully merged")

# ---------------------------------------
# 4. Create Tables If They Don’t Exist
# ---------------------------------------

cnx = mysql.connector.connect(
    host=MYSQL_HOST,
    user=MYSQL_USER,
    password=MYSQL_PASSWORD,
    database=MYSQL_DATABASE,
    port=MYSQL_PORT
)

cursor = cnx.cursor()


# table_schemas = {
#     "executive_report_kpi": """
#         CREATE TABLE IF NOT EXISTS executive_report_kpi (
#             Year INT,
#             Total_Sales DECIMAL(12,2),
#             Total_Orders INT,
#             Total_Profit DECIMAL(12,2),
#             Return_Count INT,
#             Return_Rate FLOAT,
#             Sales_YoY_Growth FLOAT,
#             Profit_YoY_Growth FLOAT
#         )
#     """,
#     "executive_report_region": """
#         CREATE TABLE IF NOT EXISTS executive_report_region (
#             Region VARCHAR(255),
#             Region_Sales DECIMAL(12,2),
#             Region_Orders INT,
#             Region_Profit DECIMAL(12,2),
#             Profit_Margin FLOAT,
#             Return_Rate FLOAT
#         )
#     """,
#     "operational_report": """
#         CREATE TABLE IF NOT EXISTS operational_report (
#             Region VARCHAR(255),
#             State VARCHAR(255),
#             City VARCHAR(255),
#             Sales_Per_Person DECIMAL(12,2),
#             Item_Unit_Price DECIMAL(12,2),
#             Quantity_Sold INT,
#             Discount_Applied FLOAT,
#             Discount_Cap FLOAT,
#             Gross_Sales DECIMAL(12,2)
#         )
#     """
# }

# for table_name, schema in table_schemas.items():
#     cursor.execute(schema)
#     print(f"✅ Table {table_name} checked/created.")

# cnx.commit()

# ---------------------------------------
# 5. Executive Report - KPIs (Yearly)
# ---------------------------------------

yearly_kpis = df_merged.groupby("Year").agg(
    Total_Sales=("Revenue Per Order", "sum"),
    Total_Orders=("Order ID", "count"),
    Return_Count=("Returned", lambda x: (x == "Yes").sum())
)

# Compute Return Rate
yearly_kpis["Return_Rate"] = (yearly_kpis["Return_Count"] / yearly_kpis["Total_Orders"]) * 100

# Compute YoY Growth
yearly_kpis["Sales_YoY_Growth"] = yearly_kpis["Total_Sales"].pct_change() * 100

# Placeholder for profit calculation
yearly_kpis["Total_Profit"] = yearly_kpis["Total_Sales"] * 0.15
yearly_kpis["Profit_YoY_Growth"] = yearly_kpis["Total_Profit"].pct_change() * 100

print("✅ Executive Report - KPIs Computed")

# ---------------------------------------
# 6. Executive Report - Regional Summary
# ---------------------------------------

regional_summary = df_merged.groupby("Region").agg(
    Region_Sales=("Revenue Per Order", "sum"),
    Region_Orders=("Order ID", "count"),
    Region_Profit=("Revenue Per Order", lambda x: x.sum() * 0.15)
)

regional_summary["Profit_Margin"] = (regional_summary["Region_Profit"] / regional_summary["Region_Sales"]) * 100
regional_summary["Return_Rate"] = (
    df_merged[df_merged["Returned"] == "Yes"].groupby("Region")["Order ID"].count()
    / regional_summary["Region_Orders"] * 100
).fillna(0)

print("✅ Executive Report - Regional Summary Computed")

# ---------------------------------------
# 7. Operational Report
# ---------------------------------------

operational_report = df_merged.groupby(["Region", "State", "City"]).agg(
    Sales_Per_Person=("Revenue Per Order", "sum"),
    Item_Unit_Price=("Revenue Per Order", "mean"),
    Quantity_Sold=("Revenue Per Order", "count"),
    Discount_Applied=("Revenue Per Order", "mean"),
    Discount_Cap=("Revenue Per Order", "max"),
    Gross_Sales=("Revenue Per Order", "sum")
)

print("✅ Operational Report Computed")

# # ---------------------------------------
# # 8. Insert One Test Row Before Full Insertion
# # ---------------------------------------

# # Ensure the test data matches the exact column count and order
# test_data = {
#     "executive_report_kpi": (2025, 100000.00, 500, 15000.00, 50, 10.5, 5.2, 3.8),  # Matches 8 columns
#     "executive_report_region": ("West", 500000.00, 2000, 75000.00, 15.0, 7.2),  # Matches 6 columns
#     "operational_report": ("West", "California", "Los Angeles", 2500.00, 300.00, 150, 10.0, 20.0, 37500.00)  # Matches 9 columns
# }

# for table, values in test_data.items():
#     # Ensure column names are wrapped correctly and match MySQL syntax
#     if table == "executive_report_kpi":
#         columns = ", ".join(["`Year`", "`Total_Sales`", "`Total_Orders`", "`Total_Profit`", "`Return_Count`", "`Return_Rate`", "`Sales_YoY_Growth`", "`Profit_YoY_Growth`"])
#     elif table == "executive_report_region":
#         columns = ", ".join(["`Region`", "`Region_Sales`", "`Region_Orders`", "`Region_Profit`", "`Profit_Margin`", "`Return_Rate`"])
#     elif table == "operational_report":
#         columns = ", ".join(["`Region`", "`State`", "`City`", "`Sales_Per_Person`", "`Item_Unit_Price`", "`Quantity_Sold`", "`Discount_Applied`", "`Discount_Cap`", "`Gross_Sales`"])

#     # Generate placeholders for values
#     placeholders = ", ".join(["%s"] * len(values))

#     # Insert data into the table
#     insert_query = f"INSERT INTO {table} ({columns}) VALUES ({placeholders})"

#     cursor.execute(insert_query, values)
#     cnx.commit()
#     print(f"✅ Test row inserted into {table}")

# ---------------------------------------
# 9. Insert All Data into MySQL Tables
# ---------------------------------------

def insert_full_data(cursor, cnx, df, table_name):
    """
    Inserts all rows from a Pandas DataFrame into a MySQL table.
    Ensures correct column order and formatting.
    """
    # Define column names dynamically based on table
    if table_name == "executive_report_kpi":
        columns = ["Year", "Total_Sales", "Total_Orders", "Total_Profit", "Return_Count", "Return_Rate", "Sales_YoY_Growth", "Profit_YoY_Growth"]
    elif table_name == "executive_report_region":
        columns = ["Region", "Region_Sales", "Region_Orders", "Region_Profit", "Profit_Margin", "Return_Rate"]
    elif table_name == "operational_report":
        columns = ["Region", "State", "City", "Sales_Per_Person", "Item_Unit_Price", "Quantity_Sold", "Discount_Applied", "Discount_Cap", "Gross_Sales"]
    else:
        raise ValueError("Invalid table name!")

    # Format column names for SQL (wrap in backticks)
    columns_sql = ", ".join([f"`{col}`" for col in columns])

    # Generate placeholders for values
    placeholders = ", ".join(["%s"] * len(columns))

    # Prepare SQL INSERT query
    insert_query = f"INSERT INTO {table_name} ({columns_sql}) VALUES ({placeholders})"

    # Convert DataFrame to list of tuples for batch insertion
    data = [tuple(row[col] for col in columns) for _, row in df.iterrows()]

    # Insert data
    cursor.executemany(insert_query, data)
    cnx.commit()
    print(f"✅ Successfully inserted {len(data)} rows into {table_name}")

# print(yearly_kpis.head())
# print(yearly_kpis.dtypes)  # Shows the data types of all columns

# Fill NaN values with 0 to avoid MySQL insertion issues
yearly_kpis.fillna(0, inplace=True)
regional_summary.fillna(0, inplace=True)
operational_report.fillna(0, inplace=True)

# Ensure "first column" is a normal column, not an index
yearly_kpis.reset_index(inplace=True)
regional_summary.reset_index(inplace=True)
operational_report.reset_index(inplace=True)

# Run insertion for all reports
insert_full_data(cursor, cnx, yearly_kpis, "executive_report_kpi")
insert_full_data(cursor, cnx, regional_summary, "executive_report_region")
insert_full_data(cursor, cnx, operational_report, "operational_report")

# Close connection
cursor.close()
cnx.close()

print("✅ All reports successfully inserted into MySQL!")


# ---------------------------------------
# 10. Save Reports as CSV Files
# ---------------------------------------

yearly_kpis.to_csv("executive_report_kpi.csv", index=True)
regional_summary.to_csv("executive_report_region.csv", index=True)
operational_report.to_csv("operational_report.csv", index=True)

print("✅ Reports exported as CSV files.")


✅ Data successfully loaded from MySQL
✅ Data successfully merged


/var/folders/x5/qny00fhx4hscgjk857bvpk5w0000gn/T/ipykernel_56071/2023063113.py:81: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_merged["Returned"].fillna("No", inplace=True)


✅ Executive Report - KPIs Computed
✅ Executive Report - Regional Summary Computed
✅ Operational Report Computed
✅ Successfully inserted 4 rows into executive_report_kpi
✅ Successfully inserted 4 rows into executive_report_region
✅ Successfully inserted 603 rows into operational_report
✅ All reports successfully inserted into MySQL!
✅ Reports exported as CSV files.
